In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
# Define input sentences

sentences = [
    "I love my dog.",
    "i love my cat.",
    "he loves dog!"
]

# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words=10)

# Generate indices for each word in the corpus
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
print(word_index)
sequences = tokenizer.texts_to_sequences(sentences)

padded = pad_sequences(sequences, padding='post')
print(padded)

{'i': 1, 'love': 2, 'my': 3, 'dog': 4, 'cat': 5, 'he': 6, 'loves': 7}
[[1 2 3 4]
 [1 2 3 5]
 [6 7 4 0]]


# Tokenizing the sarcasm dataset

In [15]:
!wget https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json

--2022-04-12 08:37:09--  https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 2607:f8b0:400f:803::2010, 2607:f8b0:400f:804::2010, 2607:f8b0:400f:807::2010, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2607:f8b0:400f:803::2010|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘sarcasm.json’

sarcasm.json        100%[===================>]   5.38M  10.0MB/s    in 0.5s    

2022-04-12 08:37:10 (10.0 MB/s) - ‘sarcasm.json’ saved [5643545/5643545]



In [19]:
import json

with open("./sarcasm.json", 'r') as f:
    datastore  = json.load(f)

In [25]:
print("Non-sarcastic headline:\n",datastore[0])
print("\nSarcastic headline:\n",datastore[20000])

Non-sarcastic headline:
 {'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5', 'headline': "former versace store clerk sues over secret 'black code' for minority shoppers", 'is_sarcastic': 0}

Sarcastic headline:
 {'article_link': 'https://www.theonion.com/pediatricians-announce-2011-newborns-are-ugliest-babies-1819572977', 'headline': 'pediatricians announce 2011 newborns are ugliest babies in 30 years', 'is_sarcastic': 1}


In [30]:
sentences=[]
labels=[]
urls=[]

for row in datastore:
    sentences.append(row['headline'])
    labels.append(row['is_sarcastic'])
    urls.append(row['article_link'])


## Pre-processing the headlines

In [31]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [42]:
# Initialize the Tokenizer class
tokenizer = Tokenizer(oov_token='<OOV>')

# Generate the word index dictionary
tokenizer.fit_on_texts(sentences)

# Print the length of the word index
word_index = tokenizer.word_index
print(f"Number of words in word_index: {len(word_index)}")

# # Print the word index
# print("word_index", word_index)

# Generate and pad the sequences
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, padding='post')

# Print sample headline
index = 2
print("Sample headline:\n", sentences[index])
print("Padded sequence: \n", padded[index])

# Print dimensions of padded sequences
print("Shape of padded sequences: ",padded.shape)

Number of words in word_index: 29657
Sample headline:
 mom starting to fear son's web series closest thing she will have to grandchild
Padded sequence: 
 [  145   838     2   907  1749  2093   582  4719   221   143    39    46
     2 10736     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0]
Shape of padded sequences:  (26709, 40)
